In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pickle
import time
import scipy.io as sio

from sklearn.metrics import r2_score

print(time.strftime('%H:%M:%S  ',time.localtime(time.time())), end="")
print("Loading data")

#####   Loading multiple data files with sio    ######

matfile1 = sio.loadmat('data_case300_1440K/data_case300_feasible_530_train.mat')
matfile2 = sio.loadmat('data_case300_1440K/data_case300_feasible_530_test.mat')
feasible_data = 0.95*np.concatenate([matfile1['feasible_data'], matfile2['feasible_data']], axis=1)
feasible_labels = -1 * np.ones(feasible_data.shape[1], dtype=np.int8)
del matfile1
del matfile2

matfile1 = sio.loadmat('data_case300_1440K/data_case300_infeasible_530_train.mat')
matfile2 = sio.loadmat('data_case300_1440K/data_case300_infeasible_530_test.mat')
infeasible_data = 1.05*np.concatenate([matfile1['infeasible_data'], matfile2['infeasible_data']], axis=1)
infeasible_labels = np.ones(infeasible_data.shape[1], dtype=np.int8)
del matfile1
del matfile2

data = np.concatenate([feasible_data, infeasible_data], axis=1).T
labels = np.concatenate([feasible_labels, infeasible_labels]).reshape(-1, 1)
del feasible_data 
del infeasible_data
del feasible_labels
del infeasible_labels

print(data.shape, labels.shape)

train_n = 1000000 # number of training samples
total_n = 1440000 # number of samples in total
train_n_c = train_n // 2
total_n_c = total_n // 2

train_i = np.array([np.arange(0, train_n_c),np.arange(0, train_n_c) + total_n_c]).T.reshape(-1)
val_i = np.array([np.arange(train_n_c, total_n_c), np.arange(train_n_c, total_n_c)+  total_n_c]).T.reshape(-1)

train_data = data[train_i, :]
train_labels = labels[train_i, :]
val_data = data[val_i, :]
val_labels = labels[val_i, :]
del data
del labels

mean_x = 0
std_x = 1

mean_x = np.mean(train_data, axis = 0)
std_x = np.std(train_data, axis = 0)
std_x[np.less(std_x, 0.001)] = 0.001


print(time.strftime('%H:%M:%S  ',time.localtime(time.time())), end="")
print("Finished data")

seed_difference = 0
if seed_difference != 0:
    print(f'***************************')
    print(f'***** SEED DIFF: {seed_difference} ********')
    print(f'***************************')

In [ ]:
%matplotlib notebook
# Train a base model on the binary data set

datafile_prefix = 'data_case300_1440K/'

hyper_param_template={'traindatasetSize' : 1000000, 'neurons' : [1024, 1],
'learningRate1' : 0.01, 'learningRate2' : 0.01, 'momentum' : 0.9,
'trainBatchSize' : 5000, 'trainingEpochs' : 2000, 'learningRateChangeAfter' : 1.0,
'valBatchSize' : 1000, 'displayPerEpochs' : 50, 'valPerEpochs' : 50,
'savePerEpochs' : 99999500, 'Regularizer' : 'L2', 'lambda' : 0.0, 'dropout_keepprob' : 1.0}

hyper_param_list = [hyper_param_template.copy() for i in range(1)]

prefixPath = os.path.join(os.getcwd(), 'models') + '/'

loadFilename = None

def bn_layer(x, scope, is_training, epsilon=0.001, decay=0.99, reuse=None):
    with tf.variable_scope(scope, reuse=reuse):
        shape = x.get_shape().as_list()
        # gamma: a trainable scale factor
        gamma = tf.get_variable("gamma", shape[-1], initializer=tf.constant_initializer(1.0), trainable=True)
        # beta: a trainable shift value
        beta = tf.get_variable("beta", shape[-1], initializer=tf.constant_initializer(0.0), trainable=True)
        moving_avg = tf.get_variable("moving_avg", shape[-1], initializer=tf.constant_initializer(0.0), trainable=False)
        moving_var = tf.get_variable("moving_var", shape[-1], initializer=tf.constant_initializer(1.0), trainable=False)
        if is_training:
            # tf.nn.moments == Calculate the mean and the variance of the tensor x
            avg, var = tf.nn.moments(x, np.arange(len(shape)-1), keep_dims=True)
            avg=tf.reshape(avg, [avg.shape.as_list()[-1]])
            var=tf.reshape(var, [var.shape.as_list()[-1]])
            #update_moving_avg = moving_averages.assign_moving_average(moving_avg, avg, decay)
            update_moving_avg=tf.assign(moving_avg, moving_avg*decay+avg*(1-decay))
            #update_moving_var = moving_averages.assign_moving_average(moving_var, var, decay)
            update_moving_var=tf.assign(moving_var, moving_var*decay+var*(1-decay))
            control_inputs = [update_moving_avg, update_moving_var]
            with tf.control_dependencies(control_inputs):
                output = tf.nn.batch_normalization(x, avg, var, offset=beta, scale=gamma, variance_epsilon=epsilon)
        else:
            avg = moving_avg
            var = moving_var
            output = tf.nn.batch_normalization(x, avg, var, offset=beta, scale=gamma, variance_epsilon=epsilon)

    return output


def bn_layer_top(x, scope, is_training, epsilon=0.001, decay=0.99):
    if is_training:
        return bn_layer(x=x, scope=scope, epsilon=epsilon, decay=decay, is_training=True, reuse=None)
    else:
        return bn_layer(x=x, scope=scope, epsilon=epsilon, decay=decay, is_training=False, reuse=True)

def getScores(sess, layer, data):
    dataSize = data.shape[0]
    print(dataSize)
    scores = []
    for i in range(0, dataSize):
        x = [data[i]]
        p = sess.run(layer, feed_dict={X_test:x, dropout_keepprob:1.0})
        scores.append(p)
    scores = np.array(scores).reshape(dataSize, -1)
    return scores

def validation(sess, val_x_data, val_y_data, hyper_param, getLoss=False, computeMean=True):
    valBatchSize = hyper_param['valBatchSize']
    valDataSize = val_x_data.shape[0]
    accu = np.zeros(valDataSize//valBatchSize)
    loss = np.zeros(valDataSize//valBatchSize)
    for i in range(0, valDataSize, valBatchSize):
        val_x = val_x_data[i:i+valBatchSize]
        val_y = val_y_data[i:i+valBatchSize]
        accu[i//valBatchSize], loss[i//valBatchSize] = sess.run([accuracy, cost_val], feed_dict={X_val:val_x, Y_val:val_y, dropout_keepprob:1.0})

    if computeMean is True:
        if getLoss is True:
            return accu.mean(), loss.mean()
        else:
            return accu.mean()
    elif getLoss is True:
        return accu, loss
    else:
        return accu

def train(sess, train_op, train_x_data, train_y_data,  val_x_data, val_y_data, hyper_param):
    global itersInTotal, totalTrainingIters, global_trainRecord, global_valRecord, global_testRecord
    
    datasetSize = hyper_param['traindatasetSize']
    learningRate1 = hyper_param['learningRate1']
    learningRate2 = hyper_param['learningRate2']
    trainBatchSize = hyper_param['trainBatchSize']
    trainingEpochs = hyper_param['trainingEpochs']
    trainingIters = hyper_param['traindatasetSize'] * trainingEpochs / trainBatchSize
    learningRateChangeAfter = int(hyper_param['learningRateChangeAfter'] * trainingIters)
    displayPerIters = hyper_param['displayPerEpochs'] * datasetSize / trainBatchSize
    valPerIters = hyper_param['valPerEpochs'] * datasetSize / trainBatchSize
    savePerIters = hyper_param['savePerEpochs'] * datasetSize / trainBatchSize
    neurons = hyper_param['neurons']
    keepprob = hyper_param['dropout_keepprob']

    iters = 0 * train_x_data.shape[0] / trainBatchSize
    trainDataSize = train_x_data.shape[0]
    learningRate = learningRate1
#     modelFilename = prefixPath + 'model_1L_' + str(neurons[0]) + 'Ns_BZ' + str(trainBatchSize) +'_SZ%dK_%depochs_reduced.ckpt'

#    pkl = open('lossRecord_1L_512Ns_BZ200_SZ120K.pkl','rb')
#    global_trainRecord = pickle.load(pkl)
#    global_valRecord = pickle.load(pkl)
#    pkl.close()
    
    f0 = plt.figure(0, figsize=(12,6))
    ax0 = f0.gca()
    ax0.plot([0])
    f0.canvas.draw()

    print(time.strftime('%H:%M:%S  ',time.localtime(time.time())), end="")
    print("Start training")
    
    for epoch in range(trainingEpochs):
        for i in range(0, trainDataSize, trainBatchSize):
            if iters > trainingIters:
                break
            iters = iters + 1
            itersInTotal = itersInTotal + 1
            train_x = train_x_data[i:i+trainBatchSize]
            train_y = train_y_data[i:i+trainBatchSize]
            _, c = sess.run([train_op, cost], feed_dict={X:train_x, Y:train_y, learning_rate:learningRate, dropout_keepprob:keepprob})
            if iters % displayPerIters == 0:
                ETA = (time.time() - beginTime) / (itersInTotal / totalTrainingIters) * (1 - itersInTotal / totalTrainingIters) / 60
                print(time.strftime('%H:%M:%S',time.localtime(time.time())), 'Est %.1fmins Elaps %.1fmins '%(ETA,(time.time() - beginTime) / 60), end="")
                print("Iters %d training cost: %f." % (iters, c))
                
            if iters % valPerIters == 0:
                print(time.strftime('%H:%M:%S  ',time.localtime(time.time())),end="")
                ret = validation(sess, train_x_data, train_y_data, hyper_param, getLoss=True)
                global_trainRecord.append([iters, ret[0], ret[1]])
                ret = validation(sess, val_x_data, val_y_data, hyper_param, getLoss=True)
                print("   Iters %d (epochs %.1f) validation loss: %f accuracy: %f" % (iters, iters*trainBatchSize/trainDataSize, ret[1], ret[0]))
                global_valRecord.append([iters, ret[0], ret[1]])
                
                ax0.cla()
                ax0.plot(np.array(global_trainRecord)[:,0], np.array(global_trainRecord)[:,2], 'r', label="Train")
                ax0.plot(np.array(global_valRecord)[:,0], np.array(global_valRecord)[:,2], 'b', label="Val")
                ax0.legend()
                ax0.grid(True)
                f0.canvas.draw()
    return


beginTime = time.time()
totalTrainingIters = hyper_param_list[-1]['traindatasetSize'] * hyper_param_list[-1]['trainingEpochs'] / hyper_param_list[-1]['trainBatchSize'] * len(hyper_param_list)
itersInTotal = 0
inputdimension = train_data.shape[1]

trainingAccuracy = np.zeros(len(hyper_param_list))
testingAccuracy = np.zeros(len(hyper_param_list))
trainingLoss = np.zeros(len(hyper_param_list))
testingLoss = np.zeros(len(hyper_param_list))


for hyper_param in hyper_param_list:
    tf.reset_default_graph()
    tf.set_random_seed(20200224+seed_difference)

    datasetSize = hyper_param['traindatasetSize']
    trainBatchSize = hyper_param['trainBatchSize']
    valBatchSize = hyper_param['valBatchSize']
    trainingEpochs = hyper_param['trainingEpochs']
    neurons = hyper_param['neurons']
    momentum = hyper_param['momentum']
    Regularizer = hyper_param['Regularizer']
    lamb = hyper_param['lambda']


    train_x_data = train_data[:datasetSize, :]
    train_y_data = train_labels[:datasetSize, :]
    val_x_data = val_data[:, :]
    val_y_data = val_labels[:, :]

    print('Train set size:', len(train_x_data), '  Val set size:', len(val_x_data))

    mean_x = 0
    std_x = 1

    mean_x = np.mean(train_x_data, axis = 0)
    std_x = np.std(train_x_data, axis = 0)
    std_x[np.less(std_x, 0.001)] = 0.001

    train_x_data = (train_x_data - mean_x) / std_x
    val_x_data = (val_x_data - mean_x) / std_x
#     test_x_data = (test_data - mean_x) / std_x
#    test_x_data = (test_x_data - mean_x) / std_x

    X = tf.placeholder(tf.float32, shape=(trainBatchSize, inputdimension), name='X')
    Y = tf.placeholder(tf.float32, shape=(trainBatchSize, neurons[-1]), name='Y')
    X_val = tf.placeholder(tf.float32, shape=(valBatchSize, inputdimension), name='X_val')
    Y_val = tf.placeholder(tf.float32, shape=(valBatchSize, neurons[-1]), name='Y_val')
    X_test = tf.placeholder(tf.float32, shape=(1, inputdimension), name='X_test')

    weights = {}
    biases = {}
    neuronsofLastLayer = inputdimension
    for i, n in enumerate(neurons):
        key = 'fc' + str(i+1)
        if (i==1):
            key='fc2_cla'
        print(key)
        weights.update({key: tf.Variable(tf.truncated_normal([neuronsofLastLayer, neurons[i]], stddev=np.sqrt(2./neuronsofLastLayer)), name='weights_'+key)})
        biases.update( {key: tf.Variable(tf.truncated_normal([neurons[i]], stddev=0.01), name='biases_'+key)})
        neuronsofLastLayer = neurons[i]

    dropout_keepprob = tf.placeholder(tf.float32, shape=[])
    def postprocessing(layer, scope='Layer', is_training=False):
#         with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
        t = tf.nn.relu(layer)
#         t = bn_layer_top(t, scope, is_training)
        # t = tf.nn.dropout(t, dropout_keepprob)
        # t = tf.layers.batch_normalization(inputs=t, name = name+'_bn', training=training)
        return t

    layers = []
    layers_val = []
    layers_test = []
    if len(neurons) == 1:
        outputLayer = tf.matmul(X, weights[key]) + biases[key]
        outputLayer_val = tf.matmul(X_val, weights[key]) + biases[key]
    else:
        for i, n in enumerate(neurons):
            key = 'fc' + str(i+1)
            if (i == 1):
                key = 'fc2_cla'
            if i == 0:
                layers.append(postprocessing(tf.matmul(X, weights[key]) + biases[key], scope=key, is_training=True))
                layers_val.append(postprocessing(tf.matmul(X_val, weights[key]) + biases[key], scope=key))
                layers_test.append(postprocessing(tf.matmul(X_test, weights[key]) + biases[key], scope=key))
            elif i == len(neurons)-1:
                outputLayer = tf.matmul(layers[-1], weights[key]) + biases[key]
                outputLayer_val = tf.matmul(layers_val[-1], weights[key]) + biases[key]
                outputLayer_test = tf.matmul(layers_test[-1], weights[key]) + biases[key]
            else:
                layers.append(postprocessing(tf.add(tf.matmul(layers[-1], weights[key]), biases[key]), scope=key, is_training=True))
                layers_val.append(postprocessing(tf.add(tf.matmul(layers_val[-1], weights[key]), biases[key]), scope=key))
                layers_test.append(postprocessing(tf.add(tf.matmul(layers_test[-1], weights[key]), biases[key], scope=key)))

    def hinge_loss(pred, groundtruth):
        return tf.reduce_mean(tf.maximum(0., 1. - pred * groundtruth))

    def regularizer(W):
        if Regularizer == 'L1':
            return lamb * tf.reduce_mean(tf.abs(W))
        elif Regularizer == 'L2':
            return lamb * tf.reduce_mean(tf.square(W))

    cost = hinge_loss(outputLayer, Y)# + regularizer(weights['fc2_cla']) + regularizer(weights['fc1'])
    cost_val = hinge_loss(outputLayer_val, Y_val)

    correct_pred = tf.equal(tf.greater(outputLayer_val, 0), tf.greater(Y_val, 0))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    learning_rate = tf.placeholder(tf.float32, shape=[])

#     with tf.variable_scope("fc1", reuse=True):
#         fc1_beta = tf.get_variable("beta")
#         fc1_gamma = tf.get_variable("gamma")
    
    var1 = [weights['fc1'], biases['fc1'], weights['fc2_cla'], biases['fc2_cla']] #, fc1_beta, fc1_gamma]
    # var2 = [weights['fc2_cla'], biases['fc2_cla']]
    # train_op1 = tf.train.MomentumOptimizer(learning_rate, momentum, use_nesterov=True).minimize(cost, var_list=var1)
    # train_op2 = tf.train.MomentumOptimizer(learning_rate, momentum, use_nesterov=True).minimize(cost, var_list=var2)
    # train_op = tf.group(train_op1, train_op2)
#     train_op = tf.train.MomentumOptimizer(learning_rate, momentum, use_nesterov=True).minimize(cost, var_list=var1)

    # train_op = tf.train.MomentumOptimizer(learning_rate, momentum, use_nesterov=True).minimize(cost)
    train_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost, var_list=var1)

    sess = tf.Session()
    saver = tf.train.Saver(var1)
    # saver = tf.train.Saver()

    if loadFilename is not None:
#         saver.restore(sess, prefixPath+loadFilename)
        init = tf.global_variables_initializer()
        saver.restore(sess, 'saved_models/1024_530')
        print(time.strftime('%H:%M:%S  ',time.localtime(time.time())),end="")
        print('Pre-trained model loaded.')
    else:
        init = tf.global_variables_initializer()
        sess.run(init)

    print('Accuracy before training: ', validation(sess, val_x_data, val_y_data, hyper_param, getLoss=True))

    global_trainRecord = []
    global_valRecord = []
    global_testRecord = []
    train_cost_history = train(sess, train_op, train_x_data, train_y_data, val_x_data, val_y_data, hyper_param)

    idx = hyper_param_list.index(hyper_param)
    trainingAccuracy[idx], trainingLoss[idx] = validation(sess, train_x_data, train_y_data, hyper_param, getLoss=True)
    testingAccuracy[idx], testingLoss[idx] = validation(sess, val_x_data, val_y_data, hyper_param, getLoss=True)

    print('network:', neurons, 'dataset:', datafile_prefix, "size:", datasetSize)
    print("train accuracy %.6f, validation accuracy %.6f"%(trainingAccuracy[idx], testingAccuracy[idx]))
    print("train loss %.6f, validation loss %.6f"%(trainingLoss[idx], testingLoss[idx]))



In [ ]:
# Save base model
saver.save(sess, 'saved_models/1024_530_current')

In [ ]:
# Load margin data

matfile = sio.loadmat('data_case300_11K_margin/data_case300_margin_ready_530_train.mat')
margin_data = matfile['margin_data'].T
boundary_data = matfile['boundary_data'].T
matfile = sio.loadmat('data_case300_11K_margin/data_case300_distance_train.mat')
dis2boundary = matfile['distance'].T
margin_data = (margin_data - mean_x) / std_x
boundary_data = (boundary_data - mean_x) / std_x
delta_data = boundary_data - margin_data

matfile = sio.loadmat('data_case300_11K_margin/data_case300_margin_ready_530_val.mat')
reg_val_x_data = matfile['margin_data'].T[:1400]
reg_val_x_data =  (reg_val_x_data - mean_x) / std_x
matfile = sio.loadmat('data_case300_11K_margin/data_case300_distance_val.mat')
reg_val_y_data = matfile['distance'].T[:1400]

In [ ]:
# Train a transfer learning model on the margin data set

def bn_layer(x, scope, is_training, epsilon=0.001, decay=0.99, reuse=None):
    with tf.variable_scope(scope, reuse=reuse):
        shape = x.get_shape().as_list()
        # gamma: a trainable scale factor
        gamma = tf.get_variable("gamma", shape[-1], initializer=tf.constant_initializer(1.0), trainable=True)
        # beta: a trainable shift value
        beta = tf.get_variable("beta", shape[-1], initializer=tf.constant_initializer(0.0), trainable=True)
        moving_avg = tf.get_variable("moving_avg", shape[-1], initializer=tf.constant_initializer(0.0), trainable=False)
        moving_var = tf.get_variable("moving_var", shape[-1], initializer=tf.constant_initializer(1.0), trainable=False)
        if is_training:
            # tf.nn.moments == Calculate the mean and the variance of the tensor x
            avg, var = tf.nn.moments(x, np.arange(len(shape)-1), keep_dims=True)
            avg=tf.reshape(avg, [avg.shape.as_list()[-1]])
            var=tf.reshape(var, [var.shape.as_list()[-1]])
            #update_moving_avg = moving_averages.assign_moving_average(moving_avg, avg, decay)
            update_moving_avg=tf.assign(moving_avg, moving_avg*decay+avg*(1-decay))
            #update_moving_var = moving_averages.assign_moving_average(moving_var, var, decay)
            update_moving_var=tf.assign(moving_var, moving_var*decay+var*(1-decay))
            control_inputs = [update_moving_avg, update_moving_var]
            with tf.control_dependencies(control_inputs):
                output = tf.nn.batch_normalization(x, avg, var, offset=beta, scale=gamma, variance_epsilon=epsilon)
        else:
            avg = moving_avg
            var = moving_var
            output = tf.nn.batch_normalization(x, avg, var, offset=beta, scale=gamma, variance_epsilon=epsilon)

    return output


def bn_layer_top(x, scope, is_training, epsilon=0.001, decay=0.99):
    if is_training:
        return bn_layer(x=x, scope=scope, epsilon=epsilon, decay=decay, is_training=True, reuse=None)
    else:
        return bn_layer(x=x, scope=scope, epsilon=epsilon, decay=decay, is_training=False, reuse=True)

def reg_validation(sess, val_x_data, val_y_data, hyper_param):
    valBatchSize = hyper_param['valBatchSize']
    valDataSize = val_x_data.shape[0]
    loss = np.zeros(valDataSize//valBatchSize)
    for i in range(0, valDataSize, valBatchSize):
        val_x = val_x_data[i:i+valBatchSize]
        val_y = val_y_data[i:i+valBatchSize]
        loss[i//valBatchSize],out = sess.run([reg_cost_val, reg_layer_val], feed_dict={X_val:val_x, reg_Y_val:val_y})
#     plt.plot(out,'.')
#     plt.plot(val_y,'.')
#     plt.show()
    return loss.mean()

def reg_train(sess, reg_train_op, reg_cost, train_x_data, train_y_data, train_delta_data, val_x_data, val_y_data, hyper_param):
    global itersInTotal, totalTrainingIters, global_trainRecord, global_valRecord, global_testRecord
    np.random.seed(20200224+seed_difference)
    
    datasetSize = hyper_param['traindatasetSize']
    learningRate1 = hyper_param['learningRate1']
    learningRate2 = hyper_param['learningRate2']
    trainBatchSize = hyper_param['trainBatchSize']
    trainingEpochs = hyper_param['trainingEpochs']
    trainingIters = hyper_param['traindatasetSize'] * trainingEpochs / trainBatchSize
    print(trainingIters)
    learningRateChangeAfter = int(hyper_param['learningRateChangeAfter'] * trainingIters)
    displayPerIters = hyper_param['displayPerEpochs'] * datasetSize / trainBatchSize
    valPerIters = hyper_param['valPerEpochs'] * datasetSize / trainBatchSize
    savePerIters = hyper_param['savePerEpochs'] * datasetSize / trainBatchSize
    neurons = hyper_param['neurons']
    keepprob = hyper_param['dropout_keepprob']
    DA_flag = reg_hyper_param['dataAugmentation']

    iters = itersInTotal
    trainDataSize = train_x_data.shape[0]
    learningRate = learningRate1
    
    for epoch in range(trainingEpochs):
        if not DA_flag:# or epoch<5000:
            np.random.seed(20200224+seed_difference)
#         learningRate1 = learningRate1 * 0.99995
#         learningRate = learningRate1
        for i in range(0, trainDataSize, trainBatchSize):
            iters = iters + 1
            itersInTotal = itersInTotal + 1
#             if (iters > learningRateChangeAfter and learningRate != learningRate2) or (iters == learningRateChangeAfter):
#                 learningRate = learningRate2
#                 print(time.strftime('%H:%M:%S  ',time.localtime(time.time())),end="")
#                 print('Now learning rate changes to %f' % learningRate)
            factor = np.random.uniform(low=0.03, high=1.0, size=(trainBatchSize, 1))
            train_x = train_x_data[i:i+trainBatchSize] + factor * train_delta_data[i:i+trainBatchSize, :]
            train_y = (1-factor) * train_y_data[i:i+trainBatchSize]
            _, c = sess.run([reg_train_op, reg_cost], feed_dict={X:train_x, reg_Y:train_y, learning_rate:learningRate})
            if iters % displayPerIters == 0:
                ETA = (time.time() - beginTime) / (itersInTotal / totalTrainingIters) * (1 - itersInTotal / totalTrainingIters) / 60
                print(time.strftime('%H:%M:%S',time.localtime(time.time())), 'Est %.1fmins Elaps %.1fmins '%(ETA,(time.time() - beginTime) / 60), end="")
                print("Iters %d training MSE: %.6f." % (iters, c))
                
            if iters % valPerIters == 0:
                global_trainRecord.append([iters, c])
                ret = reg_validation(sess, val_x_data, val_y_data, hyper_param)
                print("           Iters %d (epochs %.1f) validation MSE: %.6f" % (iters, iters*trainBatchSize/trainDataSize, ret))
                global_valRecord.append([iters, ret])
                
    return

reg_hyper_param={'traindatasetSize' : 10000, 'neurons' : [1024, 1],
'learningRate1' : 0.001, 'learningRate2' : 0.00001, 'momentum' : 0.9,
'trainBatchSize' : 5000, 'trainingEpochs' : 89200, 'learningRateChangeAfter' : 1.4,
'valBatchSize' : 200, 'displayPerEpochs' : 200, 'valPerEpochs' : 200,
'savePerEpochs' : 99999500, 'Regularizer' : 'L2', 'lambda' : 0.0000, 'dropout_keepprob' : 1.0, 'dataAugmentation': True}

datasetSize = reg_hyper_param['traindatasetSize']
reg_train_x_data = margin_data[:datasetSize, :]
reg_train_y_data = dis2boundary[:datasetSize]
reg_train_delta_data = delta_data[:datasetSize, :]
model_type = ''
DA_flag = reg_hyper_param['dataAugmentation']

tf.reset_default_graph()


inputdimension = reg_train_x_data.shape[1]
datasetSize = reg_hyper_param['traindatasetSize']
trainBatchSize = reg_hyper_param['trainBatchSize']
valBatchSize = reg_hyper_param['valBatchSize']
trainingEpochs = reg_hyper_param['trainingEpochs']
neurons = reg_hyper_param['neurons']
# momentum = reg_hyper_param['momentum']
# Regularizer = reg_hyper_param['Regularizer']
lamb = reg_hyper_param['lambda']

X = tf.placeholder(tf.float32, shape=(trainBatchSize, inputdimension), name='X')
Y = tf.placeholder(tf.float32, shape=(trainBatchSize, neurons[-1]), name='Y')
X_val = tf.placeholder(tf.float32, shape=(valBatchSize, inputdimension), name='X_val')
Y_val = tf.placeholder(tf.float32, shape=(valBatchSize, neurons[-1]), name='Y_val')
X_test = tf.placeholder(tf.float32, shape=(1, inputdimension), name='X_test')

weights = {}
biases = {}
neuronsofLastLayer = inputdimension
for i, n in enumerate(neurons):
    key = 'fc' + str(i+1)
    if (i==1):
        key='fc2_cla'
    weights.update({key: tf.Variable(tf.truncated_normal([neuronsofLastLayer, neurons[i]], stddev=np.sqrt(2./neuronsofLastLayer), seed=20200401+seed_difference), name='weights_'+key)})
    biases.update( {key: tf.Variable(tf.truncated_normal([neurons[i]], stddev=0.0001, seed=20200401+seed_difference), name='biases_'+key)})
    neuronsofLastLayer = neurons[i]

# dropout_keepprob = tf.placeholder(tf.float32, shape=[])
def postprocessing(layer, scope='Layer', is_training=False):
#     layer = bn_layer_top(layer, scope, is_training)
    layer = tf.nn.relu(layer)
    return layer

layers = []
layers_val = []
layers_test = []
if len(neurons) == 1:
    outputLayer = tf.matmul(X, weights[key]) + biases[key]
    outputLayer_val = tf.matmul(X_val, weights[key]) + biases[key]
else:
    for i, n in enumerate(neurons):
        key = 'fc' + str(i+1)
        if (i == 1):
            key = 'fc2_cla'
        if i == 0:
            layers.append(postprocessing(tf.matmul(X, weights[key]) + biases[key], scope=key, is_training=True))
            layers_val.append(postprocessing(tf.matmul(X_val, weights[key]) + biases[key], scope=key))
            layers_test.append(postprocessing(tf.matmul(X_test, weights[key]) + biases[key], scope=key))
        elif i == len(neurons)-1:
            outputLayer = tf.matmul(layers[-1], weights[key]) + biases[key]
            outputLayer_val = tf.matmul(layers_val[-1], weights[key]) + biases[key]
            outputLayer_test = tf.matmul(layers_test[-1], weights[key]) + biases[key]
        else:
            layers.append(postprocessing(tf.add(tf.matmul(layers[-1], weights[key]), biases[key]), scope=key, is_training=True))
            layers_val.append(postprocessing(tf.add(tf.matmul(layers_val[-1], weights[key]), biases[key]), scope=key))
            layers_test.append(postprocessing(tf.add(tf.matmul(layers_test[-1], weights[key]), biases[key], scope=key)))

######### First Layer of regressor
reg_weights1 = tf.Variable(tf.truncated_normal([1024,128], stddev=0.00005, seed=20200224+seed_difference), name='weights_reg')
reg_biases1 = tf.Variable(tf.truncated_normal([128], stddev=0.00005, seed=20200224+seed_difference), name='biases_reg')
# reg_weights1 = tf.Variable(tf.truncated_normal([256,64], stddev=np.sqrt(2./256), seed=20200224), name='weights1_reg')
# reg_biases1 = tf.Variable(tf.truncated_normal([64], stddev=0.0001, seed=20200224), name='biases1_reg')

reg_layer1 = tf.matmul(layers[-1], reg_weights1) + reg_biases1
reg_layer1 = tf.nn.relu(reg_layer1); model_type += 'ReLU'
model_type += ' then '
reg_layer1 = bn_layer_top(reg_layer1, 'reg1', True); model_type += 'BN2'

reg_layer1_val = tf.matmul(layers_val[-1], reg_weights1) + reg_biases1
if 'ReLU' in model_type:
    reg_layer1_val = tf.nn.relu(reg_layer1_val)
if 'BN2' in model_type:
    reg_layer1_val = bn_layer_top(reg_layer1_val, 'reg1', False)

reg_layer1_test = tf.matmul(layers_test[-1], reg_weights1) + reg_biases1
if 'ReLU' in model_type:
    reg_layer1_test = tf.nn.relu(reg_layer1_test)
if 'BN2' in model_type:
    reg_layer1_test = bn_layer_top(reg_layer1_test, 'reg1', False)


######### Output layer of regressor
reg_weights = tf.Variable(tf.truncated_normal([128, 1], stddev=0.00005, seed=202002241+seed_difference), name='weights_reg')
reg_biases = tf.Variable(tf.truncated_normal([1], mean=0.0, stddev=0.00005, seed=202002241+seed_difference), name='biases_reg')

reg_layer = tf.matmul(reg_layer1, reg_weights) + reg_biases
reg_layer_val = tf.matmul(reg_layer1_val, reg_weights) + reg_biases
reg_layer_test = tf.matmul(reg_layer1_test, reg_weights) + reg_biases

reg_Y = tf.placeholder(tf.float32, shape=(trainBatchSize, 1), name='Y_reg')
reg_Y_val = tf.placeholder(tf.float32, shape=(valBatchSize, 1), name='Y_reg_reg')

def regularizer(W):
    return lamb * tf.reduce_mean(tf.square(W))

reg_cost = tf.reduce_mean(tf.square(tf.subtract(reg_layer, reg_Y))) # + regularizer(weights['fc1']) + regularizer(reg_weights1) + regularizer(reg_weights)
reg_cost_val = tf.reduce_mean(tf.square(tf.subtract(reg_layer_val, reg_Y_val)))

    
reg_var = [reg_weights1, reg_biases1, reg_weights, reg_biases]
reg_var.extend([weights['fc1'], biases['fc1']]) # uncomment for unfreeze layer 1

if 'BN2' in model_type:
    with tf.variable_scope("reg1", reuse=True):
        reg1_beta = tf.get_variable("beta")
        reg1_gamma = tf.get_variable("gamma")
    reg_var.extend([reg1_beta, reg1_gamma])

learning_rate = tf.placeholder(tf.float32, shape=[])
reg_train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(reg_cost, var_list=reg_var)

sess = tf.Session()
    
var1 = [weights['fc1'], biases['fc1'], weights['fc2_cla'], biases['fc2_cla']]
saver = tf.train.Saver(var1)
tf.set_random_seed(20200224+seed_difference)

init = tf.global_variables_initializer()
sess.run(init)

saver.restore(sess, 'saved_models/1024_530_current'); model_type += ', TL w/o freezing' if weights['fc1'] in reg_var else ', TL'

if DA_flag:
    model_type += ', DA'
    
if 'TL' not in model_type and weights['fc1'] not in reg_var:
    model_type += ' ** LAYER 1 WRONG SETTING **'
    
global_trainRecord = []
global_valRecord = []
global_testRecord = []
beginTime = time.time()
totalTrainingIters = 0
itersInTotal = 0
print('Training set size:', len(reg_train_x_data))
print('Validation set size:', len(reg_val_x_data))
print('Model:', model_type)

In [ ]:
%matplotlib widget
# Initialize training sequence

totalTrainingIters += reg_hyper_param['traindatasetSize'] * reg_hyper_param['trainingEpochs'] / reg_hyper_param['trainBatchSize']

print('MSE before training: ', reg_validation(sess, reg_val_x_data, reg_val_y_data, reg_hyper_param))

train_cost_history = reg_train(sess, reg_train_op, reg_cost, reg_train_x_data, reg_train_y_data, reg_train_delta_data, reg_val_x_data, reg_val_y_data, reg_hyper_param)

np.random.seed(812374917+seed_difference)
factor = np.random.uniform(low=0.03, high=1.0, size=(reg_hyper_param['traindatasetSize'], 1))
reg_train_x_data_for_test = reg_train_x_data + factor * reg_train_delta_data
reg_train_y_data_for_test = (1-factor) * reg_train_y_data
trainingLoss = reg_validation(sess, reg_train_x_data_for_test, reg_train_y_data_for_test, reg_hyper_param)
testingLoss = reg_validation(sess, reg_val_x_data, reg_val_y_data, reg_hyper_param)
print("train MSE %.2f, validation MSE %.2f" % (trainingLoss, testingLoss))

idx = np.array(global_valRecord)[:,1].argmin()
print('train MSE %.6f validation MSE %.6f @ %d epochs'%(np.array(global_trainRecord)[idx,1], np.array(global_valRecord)[idx,1], 
                                                        np.array(global_valRecord)[idx,0]/(reg_hyper_param['traindatasetSize']/reg_hyper_param['trainBatchSize'])))

In [ ]:
%matplotlib inline
# Plot training/validation curves

global global_trainRecord, global_valRecord
plt.figure(figsize=(18,6))
plt.plot(np.array(global_trainRecord)[:,0]/(reg_hyper_param['traindatasetSize']/reg_hyper_param['trainBatchSize']), np.array(global_trainRecord)[:,1], 'r', label="Train")
plt.plot(np.array(global_valRecord)[:,0]/(reg_hyper_param['traindatasetSize']/reg_hyper_param['trainBatchSize']), np.array(global_valRecord)[:,1], 'b', label="Val")
plt.legend()
plt.ylim([0,0.02])
plt.grid(True)
plt.show()

In [ ]:
def getEstimateDistance(sess, data):
    dataSize = data.shape[0]
    scores = []
    for i in range(0, dataSize):
        x = [data[i]]
        p = sess.run(reg_layer_test, feed_dict={X_test:x})
        scores.append(p)
    scores = np.array(scores).reshape(dataSize, -1)
    return scores


# reg_layer_test = tf.matmul(layers_test[-1], reg_weights) + reg_biases
t = time.time()
s = getEstimateDistance(sess, reg_val_x_data)
print(time.time() - t, 's', s.shape, 'samples')

%matplotlib inline
plt.figure(3, figsize=(6,6))
plt.plot(s, reg_val_y_data, '.')
plt.xlabel('Predicted')
plt.ylabel('Actual')
lim1 = np.min([s, reg_val_y_data])
lim2 = np.max([s, reg_val_y_data])
# plt.xlim([lim1,lim2])
# plt.ylim([lim1,lim2])
plt.grid(True)
plt.title('Transfer Learning')
plt.show()

# np.savetxt('scores.csv', s, delimiter=',')
from sklearn.metrics import r2_score
print('R2 = ', r2_score(s, reg_val_y_data))
print('RMSE = ', np.sqrt(np.mean(np.square(s-reg_val_y_data))))
print('MSE = ', np.mean(np.square(s-reg_val_y_data)))